In [1]:
console.log(require('fs').readdirSync('./node_modules'));

[ '.bin', 'aws-sdk', 'skale-engine' ]


undefined

This example uses skale-engine and the amazon s3 SDK.

We will connect to an s3 bucket and process its data.

In [16]:
var sc = require('skale-engine').context();
var AWS = require('aws-sdk');

var s3 = new AWS.S3({signatureVersion: 'v4'});
var params = {Bucket: 'skale-demo', Key: 'datasets/restaurants-ny.json'};

var restaurants = sc.lineStream(s3.getObject(params).createReadStream()).persist();

undefined

In [19]:
// await restaurants.map(line => JSON.parse(line)).first()
$$async$$ = restaurants
    .map(line => JSON.parse(line))
    .first()
    .then($$done$$);

[ { address: 
     { building: '1007',
       coord: [Object],
       street: 'Morris Park Ave',
       zipcode: '10462' },
    borough: 'Bronx',
    cuisine: 'Bakery',
    grades: [ [Object], [Object], [Object], [Object], [Object] ],
    name: 'Morris Park Bake Shop',
    restaurant_id: '30075445' } ]

In [ ]:
console.log('Hello')